In [1]:
from fastapi import FastAPI, Depends, status
from starlette.exceptions import HTTPException
from fastapi.middleware.cors import CORSMiddleware
from starlette.status import HTTP_422_UNPROCESSABLE_ENTITY

#from .api.api_v1.api import router as api_router
from core.config import ALLOWED_HOSTS, PROJECT_NAME, PROJECT_VERSION, API_PORT
from core.errors import http_422_error_handler, http_error_handler
from db.mongodb_connect import close_mongo_connection, connect_to_mongo
from db.mongodb import AsyncIOMotorClient, get_database
import asyncio

In [2]:
app = FastAPI(title=PROJECT_NAME, version = PROJECT_VERSION)

if not ALLOWED_HOSTS:
    ALLOWED_HOSTS = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=ALLOWED_HOSTS,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


app.add_event_handler("startup", connect_to_mongo)
app.add_event_handler("shutdown", close_mongo_connection)


app.add_exception_handler(HTTPException, http_error_handler)
app.add_exception_handler(HTTP_422_UNPROCESSABLE_ENTITY, http_422_error_handler)

In [3]:
from typing import Optional

from pydantic import BaseModel

from typing import Any, Dict, AnyStr, List, Union

JSONObject = Dict[AnyStr, Any]
JSONArray = List[Any]
JSONStructure = Union[JSONArray, JSONObject]

In [4]:
#import adapter_model

In [5]:
connect_to_mongo()

<coroutine object connect_to_mongo at 0x7fb40ddd29e0>

In [5]:
example_text = "Dan Will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."

In [ ]:
@app.get("/model/label", tags = ["Status"], status_code=status.HTTP_200_OK)
def get_model_status():
    return {
        "message": "get success",
        "status": "re-training",
        "step": {
            "current": 30,
            "total": 50,
        },
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [ ]:
@app.get("/model/template", tags = ["Status"], status_code=status.HTTP_200_OK)
def get_model_status():
    return {
        "message": "get success",
        "status": "re-training",
        "step": {
            "current": 30,
            "total": 50,
        },
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [ ]:
class text_label_body(BaseModel):
    text: str = example_text

In [ ]:
@app.post("/model/label", tags = ["Predict"], status_code=status.HTTP_200_OK)
def text_label(data: text_label_body):
    sen, pred, logits, logits_order = adapter_model.model_predict(adapter_model.model, data.text)
    out_Tokens = []
    for i, _ in enumerate(sen):
        predictions = []
        for j, _ in enumerate(logits_order):
            predictions.append({
                "type": logits_order[j],
                "confidence": logits[i][j],
            })
        predictions.sort(key = lambda x: x["confidence"], reverse=True)
        out_Tokens.append({
            "token": sen[i],
            "predictions": predictions,
        })
    return out_Tokens

In [ ]:
@app.post("/model/template", tags = ["Predict"], status_code=status.HTTP_200_OK)
def recommand_template(data: text_label_body):
    return {
        "predict": [
            {
                "name": "Full Payment Upon Demand",
                "confidence": 0.9
            },
            {
                "name": "Acceptance of Delivery",
                "confidence": 0.8
            },
            {
                "name": "Copyright License",
                "confidence": 0.7
            },
            {
                "name": "Demand Forecast",
                "confidence": 0.6
            },
            {
                "name": "Eat Apples",
                "confidence": 0.5
            },
        ]
    }

In [ ]:
class suggest_best_practice_body(BaseModel):
    text: str = example_text

In [ ]:
@app.post("/model/suggestion", tags = ["Predict"], status_code=status.HTTP_200_OK)
def suggest_best_practice(data: suggest_best_practice_body):
    return {
        "predict": "Not Finish Yet."
    }

In [6]:
class update_data_body(BaseModel):
    text: JSONArray = [
        {
            "text": "Eason",
            "label": "Party"
        },
        {
            "text": "will",
            "label": "O"
        },
        {
            "text": "meet",
            "label": "O"
        },
        {
            "text": "Dan",
            "label": "Party"
        },
        {
            "text": "at",
            "label": "O"
        },
        {
            "text": "2021-08-04 18:00",
            "label": "TemporalUnit"
        },
        {
            "text": ".",
            "label": "O"
        },
    ]

In [7]:
@app.post("/data/label", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_data(data: update_data_body):
    return "OK"

In [8]:
class update_template_data_body(BaseModel):
    text: str = example_text
    template: str = "IP Payment"

In [9]:
@app.post("/data/template", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_template_data(data: update_template_data_body):
    return "OK"

In [10]:
class update_suggestion_data_body(BaseModel):
    text: str = "Not Finish Yet"

In [11]:
@app.post("/data/suggestion", tags = ["Optimize Data"], status_code=status.HTTP_200_OK)
def update_suggestion_data(data: update_suggestion_data_body):
    return "OK"

In [12]:
@app.post("/model/suggestion:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_suggestion_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
    }

In [13]:
@app.post("/model/template:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_template_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
    }

In [14]:
@app.post("/model/label:retrain", tags = ["ReTrain"], status_code=status.HTTP_200_OK)
def retrain_model():
    return {
        "message": "success, start retrain.",
        "train-data-amount": 2500,
        "affect-labels": ['Party', 'Race', 'SpecialTerm', 'TemporalUnit', 'Time', 'Timezone', 'US_States']
    }

In [15]:
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=13537)
    
    

INFO:     Started server process [10728]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:13537 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://0.0.0.0:13537 (Press CTRL+C to quit)


INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK
INFO:     111.241.163.16:0 - "POST /data/label HTTP/1.0" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [10728]
INFO:uvicorn.error:Finished server process [10728]


In [16]:
db

NameError: name 'db' is not defined